In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Desktop/AI'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
df_mails = pd.read_csv('spam.csv',encoding= 'latin-1')

In [5]:
df_mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#missing value

In [6]:
df_mails.isnull().sum()  #There are no missing value except unnamed column, we dont need those cols.

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
#EDA

In [7]:
df_mails.v1.value_counts()  # We can see there are total 747 spam and 4825 ham

ham     4825
spam     747
Name: v1, dtype: int64

In [8]:
df_mails.v1.value_counts().plot(kind="bar");

In [ ]:
#Split Train & Test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df_mails["v2"] # Email
y = df_mails["v1"] # Result Ham or Spam

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=34)

In [ ]:
#Term Frequency Inverse Document

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
vectorizer = TfidfVectorizer()

In [14]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
#Train Classifier

In [15]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
#Pipeline

In [16]:
from sklearn.pipeline import Pipeline

In [17]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [ ]:
#Predict

In [18]:
predictions = text_clf.predict(X_test)

In [19]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
#Confusion Matrix

In [20]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1590    4]
 [  29  216]]


In [ ]:
#Classification Report

In [21]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1594
        spam       0.98      0.88      0.93       245

    accuracy                           0.98      1839
   macro avg       0.98      0.94      0.96      1839
weighted avg       0.98      0.98      0.98      1839



In [ ]:
#Accuracy

In [22]:
print(metrics.accuracy_score(y_test, predictions)) # we got an accuray of 98% which is really amazing.

0.9820554649265906


In [ ]:
#Test New Email

In [23]:
text_clf.predict(["Weekly Lottery Participation. Win upto $10,000."])

array(['spam'], dtype=object)

In [24]:
text_clf.predict(["Hello Sir, How are you?"])

array(['ham'], dtype=object)